In [1]:
%config Completer.use_jedi = False
%matplotlib inline

#!/usr/bin/env python
import sys
import threading
import time

import cv2 as cv
import numpy as np

from centermask.config import get_cfg
from detectron2.data import MetadataCatalog
from cv_bridge import CvBridge, CvBridgeError

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
from detectron2_ros.msg import Result
from sensor_msgs.msg import Image, RegionOfInterest, CameraInfo
import pyrealsense2 as rs2

# from detectron2_ros.msg import PersonPositions, PersonPosition
from finean_msgs.msg import PersonPositions, PersonPosition

# For sync depth and rgb
import message_filters


# Needed to update backbone registry
import centermask.modeling.backbone


# For the dataset loading
from os import listdir
from os.path import isfile, join
import csv

from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
def euler_to_rotMat(yaw, pitch, roll):
    Rz_yaw = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw),  np.cos(yaw), 0],
        [          0,            0, 1]])
    Ry_pitch = np.array([
        [ np.cos(pitch), 0, np.sin(pitch)],
        [             0, 1,             0],
        [-np.sin(pitch), 0, np.cos(pitch)]])
    Rx_roll = np.array([
        [1,            0,             0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll),  np.cos(roll)]])
    # R = RzRyRx
    rotMat = np.dot(Rz_yaw, np.dot(Ry_pitch, Rx_roll))
    return rotMat



In [3]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box.
    # NOTE: We MUST ALWAYS add +1 to calculate area when working in
    # screen coordinates, since 0,0 is the top left pixel, and w-1,h-1
    # is the bottom right pixel. If we DON'T add +1, the result is wrong.
    intersection_area = (x_right - x_left + 1) * (y_bottom - y_top + 1)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1'] + 1) * (bb1['y2'] - bb1['y1'] + 1)
    bb2_area = (bb2['x2'] - bb2['x1'] + 1) * (bb2['y2'] - bb2['y1'] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [4]:
class KTPAnalyser(object):
    def __init__(self, depth_offset=100):
        print("Initializing")
        setup_logger()

        self._bridge = CvBridge()
        self.score_thresh = 0.50
        self.removal_classes = ['person']

        self.cfg = get_cfg()
        self.cfg.merge_from_file("/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml")
        self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8 # set threshold for this model
        self.cfg.MODEL.WEIGHTS = "/root/centermask2-lite-V-39-eSE-FPN-ms-4x.pth"
        self.predictor = DefaultPredictor(self.cfg)
        self._class_names = MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]).get("thing_classes", None)
        
        self.getManualKinectIntrinsics()

        self.base_T_rgb = np.eye(4)
        self.base_T_rgb[0:3,3] = np.array([-0.27, 1.271, 0.002])
        self.base_T_rgb[0:3,0:3] = euler_to_rotMat(-3.142, -0.001, -1.651)
        self.base_T_rgb = np.linalg.inv(self.base_T_rgb)
        
        self.depth_offset = depth_offset
        
        print("Centermask2 Node Initialized")

    def convertPixelToPosition(self, depth_img, mask):
        # Get the pixel location and depth of person centroid

        mask_positions_2d = np.argwhere(mask==True)
        approx_head_thresh = np.quantile(mask_positions_2d[:,0], 0.10)
        head_pos_mask = mask_positions_2d[:,0] < approx_head_thresh

        count = (head_pos_mask == 1).sum()

        y_center, x_center = mask_positions_2d[head_pos_mask, :].sum(0)/count
        
        y_thresh_mask = (np.argwhere((mask == True) | (mask == False))[:,0] < approx_head_thresh)
        y_thresh_mask = y_thresh_mask.reshape((480,640))
        
        head_mask = mask & y_thresh_mask
        
#         plt_img = np.zeros(depth_img.shape)
#         plt_img[head_mask.astype(bool)] = 1
#         plt.imshow(plt_img)
#         return
#         y_center, x_center = np.argwhere(mask==True).sum(0)/count
#         depth = np.median(depth_img[mask.astype(bool)]) + self.depth_offset

        non_zero_depth = depth_img[head_mask.astype(bool)]
        non_zero_depth = non_zero_depth[non_zero_depth>0]
        depth = np.median(non_zero_depth) + analyser.depth_offset
#         print(depth)

        # Convert to 3D position in camera coords 
        xyz = rs2.rs2_deproject_pixel_to_point(self.intrinsics, [x_center, y_center], depth/1000.0)
        
#         print("pixel [x,y, depth]")
#         print([x_center, y_center, depth])
        # return xyz
        return xyz

    def getManualKinectIntrinsics(self):
        # D: [0.0, 0.0, 0.0, 0.0, 0.0]
        # K: [525.0, 0.0, 319.5, 0.0, 525.0, 239.5, 0.0, 0.0, 1.0]
        # R: [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
        # P: [525.0, 0.0, 319.5, 0.0, 0.0, 525.0, 239.5, 0.0, 0.0, 0.0, 1.0, 0.0]


        print("Updating camera info")

        self.intrinsics = rs2.intrinsics()
        self.intrinsics.width = 640
        self.intrinsics.height = 480
        self.intrinsics.ppx = 319.5
        self.intrinsics.ppy = 239.5
        self.intrinsics.fx = 525.0
        self.intrinsics.fy = 525.0
        self.intrinsics.model = rs2.distortion.brown_conrady

    def run(self, rgb_file, depth_file, robot_pose):

        world_T_base = np.eye(4)
        world_T_base[0:3,3] = np.array([robot_pose[0], robot_pose[1], robot_pose[2]])
        
#         print(robot_pose)
        world_T_base[0:3,0:3] = euler_to_rotMat(robot_pose[5], robot_pose[4], robot_pose[3])   
         
        # Convert images
        np_image = cv.imread(rgb_file)
        np_depth_image = cv.imread(depth_file, cv.IMREAD_ANYDEPTH)

        np_depth_image_copy = np.zeros(np_depth_image.shape)
        
        # Get the masks and objects seen
        outputs = self.predictor(np_image)
        result = outputs["instances"].to("cpu")
        
#         result_msg = self.getResult(result)
        
        # Mask out people with score greater than threshold
        class_ids = result.pred_classes if result.has("pred_classes") else None
        class_names = np.array(self._class_names)[class_ids.numpy()]        
        retain_inds = (result.scores.numpy() > self.score_thresh) & (class_names == 'person')
        result = result[retain_inds]

        num_inds = sum(retain_inds)
#         print("Number of masks: {0}".format(num_inds))

#         boxes = result.pred_boxes if result.has("pred_boxes") else None
#         for i, (x1, y1, x2, y2) in enumerate(boxes):
#             continue
    
        positions_3d_world = []
        if num_inds > 0:
            masks = np.asarray(result.pred_masks)
            for i in range(num_inds):
                mask = masks[i]
                person_position = self.convertPixelToPosition(np_depth_image, mask)
#                 print(person_position)
                np_depth_image_copy[mask] = 1
                
                # Convert to world frame                 
                head_T_person = np.eye(4)
                head_T_person[0:3,3] = [person_position[0], person_position[1], person_position[2]]
                

                world_T_head = np.dot(world_T_base, self.base_T_rgb)
                world_T_person = np.dot(world_T_head, head_T_person)

                world_xyz = world_T_person[0:3,3]
#                 print('{0. 2f},{1. 2f},{2. 2f}'.format(world_xyz[0],world_xyz[1],world_xyz[2]))
#                 print(["{0:0.2f}".format(i) for i in world_xyz])
#                 print("{0:0.2f}, {1:0.2f}, {2:0.2f}".format(world_xyz[0], world_xyz[1], world_xyz[2]))
                positions_3d_world.append(world_xyz)
        
#         plt.imshow(np_depth_image_copy)
      
#         v = Visualizer(np_image[:, :, ::-1], MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]), scale=1.2)
#         v = v.draw_instance_predictions(result)
#         img = v.get_image()[:, :, ::-1]
#         plt.imshow(img)
            
#         print("Allpositions extracted...")
        return [result, positions_3d_world]
    
    def getResult(self, predictions):

        boxes = predictions.pred_boxes if predictions.has("pred_boxes") else None

        if predictions.has("pred_masks"):
            masks = np.asarray(predictions.pred_masks)
        else:
            return

        result_msg = Result()
        result_msg.class_ids = predictions.pred_classes if predictions.has("pred_classes") else None
        result_msg.class_names = np.array(self._class_names)[result_msg.class_ids.numpy()]
        result_msg.scores = predictions.scores if predictions.has("scores") else None

        for i, (x1, y1, x2, y2) in enumerate(boxes):
            mask = np.zeros(masks[i].shape, dtype="uint8")
            mask[masks[i, :, :]]=255
            mask = self._bridge.cv2_to_imgmsg(mask)
            result_msg.masks.append(mask)

            box = RegionOfInterest()
            box.x_offset = np.uint32(x1)
            box.y_offset = np.uint32(y1)
            box.height = np.uint32(y2 - y1)
            box.width = np.uint32(x2 - x1)
            result_msg.boxes.append(box)

        return result_msg
    

    @staticmethod
    def load_param(param, default=None):
        new_param = rospy.get_param(param, default)
        rospy.loginfo("[Centermask2] %s: %s", param, new_param)
        return new_param

In [5]:
def isMatch(gt_poses, est_pose):
    min_dist = 10000
    for i in range(len(gt_poses)):
        dist = np.sqrt(np.sum(np.square(np.array(gt_poses[i][1:3], dtype=float) - est_pose[0:2])))
        min_dist = min(min_dist, dist)
    
    match = False
    if min_dist < 0.3:
        match = True
        
    return [match, min_dist]

In [6]:
# Still   
timestamp_bounds_dict = {}

timestamp_bounds_dict['Still'] = {0:[1339064317.986969085, 1339064331.038245693],
1:[1339064331.768289875, 1339064360.067097068],
2:[1339064362.503071436, 1339064366.710322787],
3:[1339064369.814646105, 1339064371.979220992],
4:[1339064375.185916521, 1339064388.595394236]}

# Rotation
timestamp_bounds_dict['Rotation'] = {0:[1339072026.143797066, 1339072036.622872171],
1:[1339072038.992151379, 1339072070.026229981],
2:[1339072070.060515464, 1339072074.762523787],
3:[1339072078.269397000, 1339072080.104408822],
4:[1339072083.473387965, 1339072097.586308065]}

# Translation
timestamp_bounds_dict['Translation'] = {0:[1339066241.703633641, 1339066253.713979618],
1:[1339066255.481776113, 1339066282.147084186],
2:[1339066282.215393574, 1339066286.951295705],
3:[1339066289.056906724, 1339066291.324039606],
4:[1339066294.858379375, 1339066308.207410858]}

# Arc
timestamp_bounds_dict['Arc'] = {0:[1339072455.526836838, 1339072466.069116626],
1:[1339072472.977563961, 1339072494.467989678],
2:[1339072496.437215469, 1339072501.409138958],
3:[1339072504.309639787, 1339072506.784084572],
4:[1339072510.552881349, 1339072524.867087399]}

In [7]:
def getTaskList(full_list, lower_t, upper_t):
    task_list = []
    for t in full_list:
        if ( float(t) <= upper_t_bound) & (float(t) >= lower_t_bound):
            task_list.append(t)
    return task_list

In [11]:
dataset_folder = "/root/KTP_Dataset_Text";
image_folder = dataset_folder + "/images";
ground_truth_folder = dataset_folder + "/ground_truth";


videos = ['Still', 'Rotation', 'Translation', 'Arc']

analyser = KTPAnalyser(depth_offset=125)

# for task in range(5):
for task in [4]:
    
    total_possible_3d_matches = 0
    total_possible_2d_matches = 0
    num_3d_matches = 0
    num_2d_matches = 0
    min_dist_errors = []
    # t_stamps_with_3d_gt = 
    num_extra = 0 
    num_missed = 0
    num_3d_not_matches = 0
        
    for vid in videos:

        lower_t_bound = timestamp_bounds_dict[vid][task][0]
        upper_t_bound = timestamp_bounds_dict[vid][task][1]

        rgb_path = image_folder + "/" + vid + "/rgb"
        depth_path = image_folder + "/" + vid + "/depth"
        odom_path = ground_truth_folder + "/" + vid + "_robot_pose.txt"

        pos3d_path = ground_truth_folder + "/" + vid + "_gt3D.txt"
        pos2d_path = ground_truth_folder + "/" + vid + "_gt2D.txt"

        rgb_files = [f for f in listdir(rgb_path) if isfile(join(rgb_path, f))]
        depth_files = [f for f in listdir(depth_path) if isfile(join(depth_path, f))]

        pose_dict = {}
        with open(odom_path, 'r') as fd:
            reader = csv.reader(fd)
            for row in reader:
                arr = row[0].split(' ')
                pose_dict[arr[0].strip(':')] = arr[1:]

        gt_3d_dict = {}
        with open(pos3d_path, 'r') as fd:
            reader = csv.reader(fd)
            for row in reader:
                if len(row) > 1:
                    row = row[0:-1]
                    t_stamp = row[0].split(' ')[0].strip(':')

                    markers = []

                    for i in range(len(row)):
                        markers.append(row[i].split(' ')[1:])

                    gt_3d_dict[t_stamp] = markers

        gt_2d_dict = {}
        with open(pos2d_path, 'r') as fd:
            reader = csv.reader(fd)
            for row in reader:
                if len(row) > 1:
                    row = row[0:-1]
                    t_stamp = row[0].split(' ')[0].strip(':')

                    markers = []

                    for i in range(len(row)):
                        markers.append(row[i].split(' ')[1:])

                    gt_2d_dict[t_stamp] = markers

        rgb_timestamps = [img.strip('.jpg') for img in rgb_files]
        depth_timestamps = [img.strip('.pgm') for img in depth_files]
        pose_timestamps = list(pose_dict.keys())

        # Check that all rgb, depth and odom timetsamps match 
        depth_ctr = 0 
        pose_ctr = 0 
        for t in rgb_timestamps:
            depth_ctr += int(t in depth_timestamps)
            pose_ctr += int(t in pose_timestamps)

        #     if int(t in depth_timestamps):
        #         print(t)

        # assert(depth_ctr == len(rgb_timestamps))
        # assert(pose_ctr == len(rgb_timestamps))

        task_timestamps = getTaskList(list(gt_3d_dict.keys()), lower_t_bound, upper_t_bound)
    

        for test_t_stamp in tqdm(task_timestamps):

                try:
                    rgb_file = rgb_path + "/" + str(test_t_stamp) + '.jpg'
                    depth_file = depth_path + "/" + str(test_t_stamp) + '.pgm'
                    odom_pose = [float(i) for i in pose_dict[test_t_stamp]]
                    gt_3d_poses = gt_3d_dict[test_t_stamp]
                    gt_2d_poses = gt_2d_dict[test_t_stamp]


                    #  Run
                    [result, poses] = analyser.run(rgb_file, depth_file, odom_pose)

                    total_possible_3d_matches += len(gt_3d_poses)
                    total_possible_2d_matches += len(gt_2d_poses)

                    # 3D position checking         
                    for pose in poses:
                        [match, min_dist] = isMatch(gt_3d_poses, pose)
                        num_3d_matches += int(match)
                        num_3d_not_matches += int(not(match))
            #             if not(match):
            #                 print([test_t_stamp, min_dist])
                        min_dist_errors.append([match, min_dist])

                    if len(poses) > len(gt_3d_poses):
                        print("Video: {0} \t t_stamp: {1}".format(vid, test_t_stamp))
                        num_extra += len(poses) - len(gt_3d_poses)

                    if len(gt_3d_poses) > len(poses):
                        num_missed += len(gt_3d_poses) - len(poses)

                    # 2D bbox checking
                    boxes = result.pred_boxes if result.has("pred_boxes") else None
                    img_matches = 0
                    for i, (x1, y1, x2, y2) in enumerate(boxes):

                                bb1 = {'x1': x1, 'x2': x2, 'y1': y1, 'y2': y2}
                                max_iou = 0
                                for inst in gt_2d_poses:
                                    gt_bb = {'x1': int(inst[1]), 
                                             'x2': int(inst[1]) + int(inst[3]), 
                                             'y1': int(inst[2]), 
                                             'y2': int(inst[2]) + int(inst[4])}

                                    iou = get_iou(bb1, gt_bb)
                                    max_iou = max(max_iou, iou)

                                img_matches += int(max_iou >= 0.5)
                    num_2d_matches += max(img_matches, len(gt_3d_poses))
                except:
                    continue

    min_dist_errors = np.array(min_dist_errors)

    #     np.array
    #     # The mean error for associated positions         
    mean_assoc_dist_error = np.mean(min_dist_errors[min_dist_errors[:, 0] == 1, 1])

    print("Task {0} \t \
          Num 3d matches:{1} \t Num 2d matches:{2} \t \
          2d rate:{3} \t 3d rate: {4} \t \
          mean_error(m): {5} \t \
          fn: {6} \t fp: {7}".format(task, 
                                      num_3d_matches, 
                                      num_2d_matches, 
                                      (num_2d_matches-num_extra-num_missed)/total_possible_2d_matches,
                                     (num_3d_matches-num_extra-num_missed)/total_possible_3d_matches,
                                      mean_assoc_dist_error,
                                    num_missed/total_possible_2d_matches,
                                     num_extra/total_possible_2d_matches
                                        ))

Initializing
WARNING [10/18 19:35:05 d2.config.compat]: Config '/root/centermask2/configs/centermask/centermask_lite_V_39_eSE_FPN_ms_4x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Updating camera info
Centermask2 Node Initialized


 22%|██▏       | 46/212 [00:03<00:12, 13.31it/s]

Video: Still 	 t_stamp: 1339064377.187062474
Video: Still 	 t_stamp: 1339064377.221175088
Video: Still 	 t_stamp: 1339064377.254599373


 23%|██▎       | 48/212 [00:03<00:12, 13.07it/s]

Video: Still 	 t_stamp: 1339064377.287785292
Video: Still 	 t_stamp: 1339064377.320802203
Video: Still 	 t_stamp: 1339064377.354114964


...

Video: Arc 	 t_stamp: 1339072518.757587414
Video: Arc 	 t_stamp: 1339072518.796500068
Video: Arc 	 t_stamp: 1339072519.225629027


 98%|█████████▊| 146/149 [00:11<00:00, 12.02it/s]

Video: Arc 	 t_stamp: 1339072520.093736325
Video: Arc 	 t_stamp: 1339072520.131320403
Video: Arc 	 t_stamp: 1339072520.161490589


100%|██████████| 149/149 [00:12<00:00, 12.40it/s]

Video: Arc 	 t_stamp: 1339072520.196616961
Video: Arc 	 t_stamp: 1339072520.230644998
Task 4 	           Num 3d matches:2163 	 Num 2d matches:2245 	           2d rate:0.7909579230080573 	 3d rate: 0.7542524619516562 	           mean_error(m): 0.19669444224384505 	           fn: 0.007162041181736795 	 fp: 0.20680393912264997


In [ ]:
task_timestamps

In [ ]:
# for i in sorted(list(gt_2d_dict.keys())):
#     print(i)

# sorted_tstamps = [float(i) for i in list(gt_2d_dict.keys())]
# sorted_tstamps = sorted(sorted_tstamps, key=float)
# plt.scatter(sorted_tstamps, np.ones(len(sorted_tstamps)))
# plt.show()
# for i in sorted_tstamps:
#     print(i)

In [ ]:

for offset in [125]:
# for offset in [0, 50, 75, 100, 125, 150]:
    analyser = KTPAnalyser(depth_offset=offset)
    total_possible_3d_matches = 0
    total_possible_2d_matches = 0
    num_3d_matches = 0
    num_2d_matches = 0
    min_dist_errors = []
    # t_stamps_with_3d_gt = 
    num_extra = 0 
    num_missed = 0
    num_3d_not_matches = 0
    # for test_t_stamp in list(gt_3d_dict.keys()):
#     for test_t_stamp in ['1339064321.890846829']:
    for test_t_stamp in tqdm(list(gt_3d_dict.keys())):

        try:
            rgb_file = rgb_path + "/" + str(test_t_stamp) + '.jpg'
            depth_file = depth_path + "/" + str(test_t_stamp) + '.pgm'
            odom_pose = [float(i) for i in pose_dict[test_t_stamp]]
            gt_3d_poses = gt_3d_dict[test_t_stamp]
            gt_2d_poses = gt_2d_dict[test_t_stamp]


            #  Run
            [result, poses] = analyser.run(rgb_file, depth_file, odom_pose)

            total_possible_3d_matches += len(gt_3d_poses)
            total_possible_2d_matches += len(gt_2d_poses)

            # 3D position checking         
            for pose in poses:
                [match, min_dist] = isMatch(gt_3d_poses, pose)
                num_3d_matches += int(match)
                num_3d_not_matches += int(not(match))
    #             if not(match):
    #                 print([test_t_stamp, min_dist])
                min_dist_errors.append([match, min_dist])

            if len(poses) > len(gt_3d_poses):
                num_extra += len(poses) - len(gt_3d_poses)

            if len(gt_3d_poses) > len(poses):
                num_missed += len(gt_3d_poses) - len(poses)

            # 2D bbox checking
            boxes = result.pred_boxes if result.has("pred_boxes") else None
            img_matches = 0
            for i, (x1, y1, x2, y2) in enumerate(boxes):

                        bb1 = {'x1': x1, 'x2': x2, 'y1': y1, 'y2': y2}
                        max_iou = 0
                        for inst in gt_2d_poses:
                            gt_bb = {'x1': int(inst[1]), 
                                     'x2': int(inst[1]) + int(inst[3]), 
                                     'y1': int(inst[2]), 
                                     'y2': int(inst[2]) + int(inst[4])}

                            iou = get_iou(bb1, gt_bb)
                            max_iou = max(max_iou, iou)

                        img_matches += int(max_iou >= 0.5)
            num_2d_matches += max(img_matches, len(gt_3d_poses))
        except:
            continue
            
    min_dist_errors = np.array(min_dist_errors)

#     np.array
#     # The mean error for associated positions         
    mean_assoc_dist_error = np.mean(min_dist_errors[min_dist_errors[:, 0] == 1, 1])
        
    print("Offset {0} \t  \
          Num 3d matches:{1} \t Num 2d matches:{2} \t \
          2d rate:{3} \t 3d rate: {4} \t \
          mean_error(m): {5} \t \
          fn: {6} \t fp: {7}".format(offset, 
                                              num_3d_matches, 
                                              num_2d_matches, 
                                              (num_2d_matches-num_extra-num_missed)/total_possible_2d_matches,
                                             (num_3d_matches-num_extra-num_missed)/total_possible_3d_matches,
                                              mean_assoc_dist_error,
                                            num_missed/total_possible_2d_matches,
                                             num_extra/total_possible_2d_matches
                                                ))       

# Task First Loop

In [ ]:
dataset_folder = "/root/KTP_Dataset_Text";
image_folder = dataset_folder + "/images";
ground_truth_folder = dataset_folder + "/ground_truth";


videos = ['Still', 'Rotation', 'Translation', 'Arc']

for vid in videos:

    rgb_path = image_folder + "/" + vid + "/rgb"
    depth_path = image_folder + "/" + vid + "/depth"
    odom_path = ground_truth_folder + "/" + vid + "_robot_pose.txt"

    pos3d_path = ground_truth_folder + "/" + vid + "_gt3D.txt"
    pos2d_path = ground_truth_folder + "/" + vid + "_gt2D.txt"

    rgb_files = [f for f in listdir(rgb_path) if isfile(join(rgb_path, f))]
    depth_files = [f for f in listdir(depth_path) if isfile(join(depth_path, f))]

    pose_dict = {}
    with open(odom_path, 'r') as fd:
        reader = csv.reader(fd)
        for row in reader:
            arr = row[0].split(' ')
            pose_dict[arr[0].strip(':')] = arr[1:]

    gt_3d_dict = {}
    with open(pos3d_path, 'r') as fd:
        reader = csv.reader(fd)
        for row in reader:
            if len(row) > 1:
                row = row[0:-1]
                t_stamp = row[0].split(' ')[0].strip(':')

                markers = []

                for i in range(len(row)):
                    markers.append(row[i].split(' ')[1:])

                gt_3d_dict[t_stamp] = markers

    gt_2d_dict = {}
    with open(pos2d_path, 'r') as fd:
        reader = csv.reader(fd)
        for row in reader:
            if len(row) > 1:
                row = row[0:-1]
                t_stamp = row[0].split(' ')[0].strip(':')

                markers = []

                for i in range(len(row)):
                    markers.append(row[i].split(' ')[1:])

                gt_2d_dict[t_stamp] = markers

    rgb_timestamps = [img.strip('.jpg') for img in rgb_files]
    depth_timestamps = [img.strip('.pgm') for img in depth_files]
    pose_timestamps = list(pose_dict.keys())

    # Check that all rgb, depth and odom timetsamps match 
    depth_ctr = 0 
    pose_ctr = 0 
    for t in rgb_timestamps:
        depth_ctr += int(t in depth_timestamps)
        pose_ctr += int(t in pose_timestamps)

        #     if int(t in depth_timestamps):
        #         print(t)

        # assert(depth_ctr == len(rgb_timestamps))
        # assert(pose_ctr == len(rgb_timestamps))
    
    for task in range(5):
        lower_t_bound = timestamp_bounds_dict[vid][task][0]
        upper_t_bound = timestamp_bounds_dict[vid][task][1]
        
        task_timestamps = getTaskList(list(gt_3d_dict.keys()), lower_t_bound, upper_t_bound)

        for offset in [125]:
        # for offset in [0, 50, 75, 100, 125, 150]:
            analyser = KTPAnalyser(depth_offset=offset)
            total_possible_3d_matches = 0
            total_possible_2d_matches = 0
            num_3d_matches = 0
            num_2d_matches = 0
            min_dist_errors = []
            # t_stamps_with_3d_gt = 
            num_extra = 0 
            num_missed = 0
            num_3d_not_matches = 0
            # for test_t_stamp in list(gt_3d_dict.keys()):
        #     for test_t_stamp in ['1339064321.890846829']:
            for test_t_stamp in tqdm(task_timestamps):

                try:
                    rgb_file = rgb_path + "/" + str(test_t_stamp) + '.jpg'
                    depth_file = depth_path + "/" + str(test_t_stamp) + '.pgm'
                    odom_pose = [float(i) for i in pose_dict[test_t_stamp]]
                    gt_3d_poses = gt_3d_dict[test_t_stamp]
                    gt_2d_poses = gt_2d_dict[test_t_stamp]


                    #  Run
                    [result, poses] = analyser.run(rgb_file, depth_file, odom_pose)

                    total_possible_3d_matches += len(gt_3d_poses)
                    total_possible_2d_matches += len(gt_2d_poses)

                    # 3D position checking         
                    for pose in poses:
                        [match, min_dist] = isMatch(gt_3d_poses, pose)
                        num_3d_matches += int(match)
                        num_3d_not_matches += int(not(match))
            #             if not(match):
            #                 print([test_t_stamp, min_dist])
                        min_dist_errors.append([match, min_dist])

                    if len(poses) > len(gt_3d_poses):
                        num_extra += len(poses) - len(gt_3d_poses)

                    if len(gt_3d_poses) > len(poses):
                        num_missed += len(gt_3d_poses) - len(poses)

                    # 2D bbox checking
                    boxes = result.pred_boxes if result.has("pred_boxes") else None
                    img_matches = 0
                    for i, (x1, y1, x2, y2) in enumerate(boxes):

                                bb1 = {'x1': x1, 'x2': x2, 'y1': y1, 'y2': y2}
                                max_iou = 0
                                for inst in gt_2d_poses:
                                    gt_bb = {'x1': int(inst[1]), 
                                             'x2': int(inst[1]) + int(inst[3]), 
                                             'y1': int(inst[2]), 
                                             'y2': int(inst[2]) + int(inst[4])}

                                    iou = get_iou(bb1, gt_bb)
                                    max_iou = max(max_iou, iou)

                                img_matches += int(max_iou >= 0.5)
                    num_2d_matches += max(img_matches, len(gt_3d_poses))
                except:
                    continue

            min_dist_errors = np.array(min_dist_errors)

        #     np.array
        #     # The mean error for associated positions         
            mean_assoc_dist_error = np.mean(min_dist_errors[min_dist_errors[:, 0] == 1, 1])

            print("Video {0} \t  Task {1}\
                  Num 3d matches:{2} \t Num 2d matches:{3} \t \
                  2d rate:{4} \t 3d rate: {5} \t \
                  mean_error(m): {6} \t \
                  fn: {7} \t fp: {8}".format(vid, task, 
                                                      num_3d_matches, 
                                                      num_2d_matches, 
                                                      (num_2d_matches-num_extra-num_missed)/total_possible_2d_matches,
                                                     (num_3d_matches-num_extra-num_missed)/total_possible_3d_matches,
                                                      mean_assoc_dist_error,
                                                    num_missed/total_possible_2d_matches,
                                                     num_extra/total_possible_2d_matches
                                                        ))       